### 📘 Lesson 5: South Africa Flexibility

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Fabian Hofmann</strong> - Senior Optimization and Energy System Modelling Expert</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  



* Introduce participants to the PyPSA toolbox.  
* Provide details of relevant components.  
* Build and solve a simple PyPSA model.  
* Review the data structures for static and time-series data.  
* Analyze the results.  

The content also includes references to other toolboxes such as `numpy`, `pandas`, `matplotlib`, and `plotly`, but only covers functions relevant to a PyPSA workflow.  

📌 Participants unfamiliar with these toolboxes are encouraged to explore online videos or courses for deeper learning. 🎥📚  

---

Modelling reserves


- Lesson 5 - SA Network - Hourly
- Lesson 6 - SA Network - 4 second (battery and gas engine)
- Lesson 7 - Hydro
- Lesson 8 - Market modelling (EU)




### 📥 **Importing Essential Libraries**  


In [1]:
import pypsa
import pandas as pd
import numpy as np
from training_scripts import *

#### **🔧⚡ Create a PyPSA Network Object**

#### **Importing Networks in PyPSA**

PyPSA allows importing networks using three primary formats: **CSV**, **HDF5**, and **NetCDF**. Each format has its corresponding function.


**1️⃣ Importing a Network from CSV**

PyPSA can load networks stored in a directory containing CSV files. This is useful for human-readable and editable data.

```python
network.import_from_csv_folder("path_to_csv_directory")
```
**2️⃣ Importing a Network from HDF5**

HDF5 is a binary format that allows for fast loading and saving of networks.

```python
network = pypsa.Network("path_to_hdf5_file.h5")
```

**3️⃣ Importing a Network from NetCDF**

NetCDF is another binary format commonly used for scientific computing.

```python
network = pypsa.Network("path_to_netcdf_file.nc")
```


#### **📝 Setting up a data import using a spreadsheet.**

In this example, a **simplified workflow** is demonstrated using an Excel spreadsheet. The **actual model** will implement a more **formal data processing** workflow, similar to the approach used in PyPSA-EUR.

* A spreadsheet has been developed that contains some of the main input data sources for a PyPSA model. 
* The spreadsheet is in `.xlsx` format can be opened in Microsoft Excel or Google Sheets. 
* A custom function is applied to convert the excel spreadsheet into a folder of csv files that can be uploaded into the PyPSA network. 


---




**⚡ Generators**




Generators attach to a single bus, converting energy from their `carrier` to the bus `carrier`.  

* Their power output is constrained by `p_nom * p_max_pu` and `p_nom * p_min_pu`.  

* Static limits define dispatchable generators, while time-varying limits model renewables.  

* Time series `p_max_pu` and `p_min_pu` determine availability per snapshot.  

* For unit commitment constraints, refer to the PyPSA documentation. 

Some key variables relevant to this model are given below. 


| Attribute              | Type            | Unit           | Default | Description | Constraint |
|------------------------|----------------|---------------|---------|-------------|------------------|
| `name`              | string         | n/a           | n/a     | Unique name |  |
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |  |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. |  |
| `p_nom_extendable` | boolean        |           | False   | Switch to allow capacity p_nom to be extended in optimization. | |
| `p_min_pu`        | static/series  | per unit      | n/a     | Minimum output per unit of p_nom. | $p_t \geq p_{nom}\times p_{min,pu,t}$ |
| `p_max_pu`        | static/series  | per unit      | 1       | Maximum output per unit of p_nom. | $p_t \leq p_{nom}\times p_{max,pu,t}$ |
| `p_set`           | static/series  | MW            | n/a     | Active power set point (for PF). | $p_t = p_{set}$  |
| `e_sum_min`       | float          | MWh           | -inf    | Minimum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `e_sum_max`       | float          | MWh           | inf     | Maximum total energy produced during optimization horizon. | $\sum p_t \cdot \delta t \leq e_{\max}$
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | |



**⏳ Set `snapshots`** 

**Buses**

**Carriers**

🌞 **Applying `p_max_pu` Constraint on VRE Generators**  

* Variable Renewable Energy (VRE) generators, such as solar and wind, have time-dependent availability limits.  
* The `p_max_pu` constraint, imported as a time-series dataset, determines the maximum power output at each snapshot on a per unit basis.  




---
🔗 **Links**



* Links enable controllable, directed power flow between two buses (`bus0 → bus1`).  
* They can have efficiency losses and marginal costs, restricting default flow to one direction.  
* For bidirectional, lossless operation, set `efficiency = 1`, `marginal_cost = 0`, and `p_min_pu = -1`.  
* Links model HVDC interconnections, converters, heat pumps, electrolysers, and other controllable power flows.  
* ⚠️ In the actual model, lines will be used instead of links for passive AC/DC transmission.  


---
🔌 **Loads**

 

* A load connects to a single bus and consumes power as a PQ load.  
* It can represent electricity demand or other types of loads like hydrogen or heat.  
* If active power is consumed, the load draws from the bus.  
* If reactive power is consumed, the load behaves like an inductor.  
* Loads are essential for demand modeling in power system simulations. ⚡🏠  


---
🔋 **Storage: `store` vs. `storage_unit`**  



There are two components for energy storage in PyPSA: Storage Units and Stores.  

* ⚡ Storage Unit  
   * Attaches to a single bus and is used for inter-temporal power shifting with a time-varying state of charge.  
   * The energy capacity is defined as `max_hours * nominal power (MW)`, and it includes charging/discharging efficiencies.  

* 🏭 Store  
   * Connects to a single bus and acts as a fundamental energy storage component without energy conversion.  
   * Controls and optimizes energy capacity size, but power output must be controlled using Link components.  

🔄 Key Differences  
| Feature           | Storage Unit | Store |
|------------------|-------------|-------|
| Power Control | Directly defined | Requires Links |
| Energy Capacity | Fixed as `max_hours * MW` | Optimized independently |
| Marginal Cost | Applies only to discharging | Applies to both charging & discharging |
| Energy Carrier Conversion| Possible | Not possible (inherits from bus) |

Stores are more flexible but require Links for power control, while Storage Units offer a simpler implementation for direct energy storage modeling. ⚙️🔄  


⚡ **Storage Unit**


| Attribute              | Type            | Unit           | Default | Description |
|------------------------|----------------|---------------|---------|-------------|
| `name`              | string         | n/a           | n/a     | Unique name | 
| `bus`               | string         | n/a           | n/a     | Name of bus to which generator is attached |
| `p_nom`            | float          | MW            | 0       | Nominal power for limits in optimization. | 
| `marginal_cost`   | static/series  | currency/MWh  | n/a     | Marginal cost of production of 1 MWh. | 
| `max_hours` | float | h | 1 | Maximum state of charge capacity in terms of hours at full output capacity `p_nom` |  
|`state_of_charge_initial`| float | MWh | 0  | State of charge before the snapshots in the OPF. | 
|`efficiency_store`  | static/series | per unit | 1 | Efficiency of storage on the way into the storage. | 
|`efficiency_dispatch`| static/series | per unit | 1 | Efficiency of storage on the way out of the storage. | 
| `standing_loss`| static/series | per unit | 0 | Losses per hour to state of charge. | 
| `inflow`| static/series | MW | 0 | Inflow to the state of charge, e.g. due to river inflow in hydro reservoir. |








---
### 🏗️ Working with the `network` object

The network contains functions, such as: 

- 📥 Adding data: `network.add()` or `network.import_from_csv()` - As described before.
- 🔍 Optimization: `network.optimize()` – Runs the optimization process.  
   * Supports multiple solvers including GLPK, Gurobi, CPLEX, and HiGHS. 
- 📊 Statistics: `network.statistics()` – Generates system-wide statistics.  
- 🗺️ Visualization: `network.plot()` – Plots the network layout.  


**Import Network**

In [17]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [21]:
input_file_name = 'data/Lesson4_solution.xlsx'
path = convert_selected_sheets_to_csv(input_file_name, 'lesson4_csv_folder')

INFO:root:Converted snapshots to CSV.
INFO:root:Converted generators-p_max_pu to CSV.
INFO:root:Converted loads-p_set to CSV.
INFO:root:Converted storage_units-inflow to CSV.
INFO:root:Conversion complete. CSV files are saved in 'lesson4_csv_folder'
INFO:root:Excel file closed successfully.


**Review Input Data**

In [19]:
path

'lesson4_csv_folder'

In [20]:
network.import_from_csv_folder(path)


ERROR:pypsa.io:Error, no buses found


In [ ]:
network.generators_t.p_max_pu.head()

In [ ]:
network.loads_t.p_set.head()

In [ ]:
network.loads

In [ ]:
network.links.head()

**Solve Model**

In [ ]:
network.optimize(solver_name='highs')

**Results**

In [ ]:
network.generators_t.p.head()

In [ ]:
network.storage_units_t.p.head()

In [ ]:
bus_to_view = "NO"

# Get generators and storage units connected to the selected bus
clustered_generators = network.generators.query("bus == @bus_to_view").index
clustered_storage_units = network.storage_units.query("bus == @bus_to_view").index

# Extract time-series data for the selected generators and storage units
gen_p = network.generators_t.p[clustered_generators]
storage_p = network.storage_units_t.p[clustered_storage_units]

# Combine both data frames (handle cases where one may be empty)
combined_df = gen_p.join(storage_p, how="outer").fillna(0)

combined_df.plot()


In [ ]:
# Get all buses in the network
buses = network.buses.index

# generator and storage unit mapping to buses
gen_bus_map = network.generators["bus"]
store_bus_map = network.storage_units["bus"]

# Initialize a DataFrame to store marginal prices per bus
marginal_price_per_bus = pd.DataFrame(index=network.snapshots, columns=buses)

# Loop through each timestep
for t in network.snapshots:
    # Initialize dictionary to store marginal prices for this timestep
    marginal_prices_t = {}

    # Get operational generators and storage units for this timestep
    operational_gens = network.generators_t.p.loc[t]
    operational_stores = network.storage_units_t.p.loc[t]

    # Loop through each bus
    for bus in buses:
        # Get generators and storage units connected to this bus
        gen_at_bus = gen_bus_map[gen_bus_map == bus].index
        store_at_bus = store_bus_map[store_bus_map == bus].index

        # Filter only active (p > 0) units
        active_gens = operational_gens[gen_at_bus][operational_gens[gen_at_bus] > 0].index
        active_stores = operational_stores[store_at_bus][operational_stores[store_at_bus] > 0].index

        # Get marginal costs of active generators and storage units
        costs_gens = network.generators.loc[active_gens, "marginal_cost"] if not active_gens.empty else pd.Series(dtype=float)
        costs_stores = network.storage_units.loc[active_stores, "marginal_cost"] if not active_stores.empty else pd.Series(dtype=float)

        # Find the highest marginal cost at this bus
        all_costs = pd.concat([costs_gens, costs_stores])
        marginal_prices_t[bus] = all_costs.max() if not all_costs.empty else 0

    # Store results in the DataFrame
    marginal_price_per_bus.loc[t] = marginal_prices_t

marginal_price_per_bus.head()

In [ ]:
marginal_price_per_bus.plot()

### 
---